 # Import the RPB model

In [ ]:
from RPB_model import *

# Create a full RPB model (Only need to run once)

In [ ]:
RPB = full_model_creation(lean_temp_connection=True, configuration = "counter-current")

# add objective
@RPB.Expression()
def obj(RPB):
    return RPB.energy_requirement

RPB.objective = Objective(expr=RPB.obj)

RPB.objective.pprint()

### Create solver object (Only need to run once)

In [ ]:
# results = SolverFactory("gams").solve(
#     RPB,
#     tee=True,
#     keepfiles=True,
#     solver="conopt4",
#     tmpdir="temp",
#     add_options=["gams_model.optfile=1;"],
# )

solver = SolverFactory("ipopt")
solver.options = {
        "max_iter": 1000,
        "bound_push": 1e-8,
        # "halt_on_ampl_error": "yes",
        "tol": 1e-4,
        "max_cpu_time": 5*60,
        # "mu_strategy": "adaptive",
        }

# Set Bounds for Decision Variables
##### Should only need to run when initializing the model

In [ ]:
RPB.ads.L.setlb(0.01)
RPB.ads.L.setub(40)
RPB.des.L.setlb(0.01)
RPB.des.L.setub(40)

RPB.ads.Tx.setlb(25+273)
RPB.ads.Tx.setub(95+273)

RPB.des.Tx.setlb(100+273)
RPB.des.Tx.setub(160+273)

RPB.ads.P_in.setub(1.5)
RPB.ads.P.setub(1.5)

RPB.des.P_in.setub(1.5)
RPB.des.P_in.setlb(1.01325)
RPB.des.P.setub(1.5)

RPB.ads.w_rpm.setlb(0.00001)
RPB.ads.w_rpm.setub(0.1)

# Initialize Using Json File (90% Capture)
## Only run first time or if solver fails to converge to an optimal solution
#### May have to scroll to see all results

In [ ]:
from_json(RPB, fname="polishing step optimized solution 031824.json.gz", gz=True)

# Run to get RPB Results
### Can copy and paste the table to excel (formatting might just be weird)

In [ ]:
report(RPB)

# Fix Capture Efficiency and Solve

In [ ]:
RPB.ads.CO2_Capture.fix(0.9)

solver.solve(RPB, tee=True)